In [36]:
import numpy as np
import optunity
import optunity.metrics

original_data = np.genfromtxt('../../../working_data/normalised_delta_updrs_sigfall.csv', delimiter=',', skip_header=True)
_, n_columns = original_data.shape

data = original_data[:,0:(n_columns - 1)]
labels = original_data[:,(n_columns - 1)]

# Common cross validator for all models
cv_decorator = optunity.cross_validated(x=data, y=labels, num_folds=10)

results = []

In [27]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

def gnb_tuned_auroc(x_train, y_train, x_test, y_test, sigfall_prior):
    no_fall_prior = 1.0 - sigfall_prior
    model = GaussianNB(priors=[no_fall_prior,sigfall_prior]).fit(x_train, y_train)
    decision_values = model.predict(x_test)
    auc = optunity.metrics.roc_auc(y_test, decision_values)
    return auc
    
    
gnb_tuned_auroc = cv_decorator(gnb_tuned_auroc)
gnb_optimal_pars, gnb_info, _ = optunity.maximize(gnb_tuned_auroc, solver_name='grid search', num_evals=100, sigfall_prior=[0.01,0.99])

print("Optimal parameters" + str(gnb_optimal_pars))
print("AUROC of tuned model: %1.3f" % gnb_info.optimum)

results.append({'model': 'Gaussian Naive Bayes',
               'Optimal parameters': gnb_optimal_pars,
               'ROC_AUC': gnb_info.optimum
               })

Optimal parameters{'sigfall_prior': 0.7695}
AUROC of tuned model: 0.683


In [24]:
# Support Vector Machines
import sklearn.svm

# Finding the optimum SVM by abstracting over possible Kernel functions:
space = {'kernel': {'linear': {'C': [0, 2], 'sigfall_class_weight':[0,25]},
                    'rbf': {'logGamma': [-5, 0], 'C': [0, 10], 'sigfall_class_weight':[0,25]},
                    'poly': {'degree': [2, 5], 'C': [0, 5], 'coef0': [0, 2], 'sigfall_class_weight':[0,25]}
                    }
         }

# Need to use conditional func because sci-kit wont take None argument types
def train_model(x_train, y_train, kernel, C, logGamma, degree, coef0, sigfall_class_weight):
    if kernel == 'linear':
        model = sklearn.svm.SVC(kernel=kernel, C=C, class_weight={1: sigfall_class_weight})
    elif kernel == 'poly':
        model = sklearn.svm.SVC(kernel=kernel, C=C, degree=degree, coef0=coef0, class_weight={1: sigfall_class_weight})
    elif kernel == 'rbf':
        model = sklearn.svm.SVC(kernel=kernel, C=C, gamma=10 ** logGamma, class_weight={1: sigfall_class_weight})
    else:
        raise ArgumentError("Unknown kernel function: %s" % kernel)
    model.fit(x_train, y_train)
    return model

def svm_tuned_auroc(x_train, y_train, x_test, y_test, kernel='linear', C=0, logGamma=0, degree=0, coef0=0, sigfall_class_weight=0):
    model = train_model(x_train, y_train, kernel, C, logGamma, degree, coef0, sigfall_class_weight)
    decision_values = model.decision_function(x_test)
    return optunity.metrics.roc_auc(y_test, decision_values)

svm_tuned_auroc = cv_decorator(svm_tuned_auroc)

svm_optimal_pars, svm_info, _ = optunity.maximize_structured(svm_tuned_auroc, space, num_evals=150)
print("Optimal parameters" + str(svm_optimal_pars))
print("AUROC of tuned SVM: %1.3f" % svm_info.optimum)

results.append({'model': 'Support Vector Machine',
               'Optimal parameters': svm_optimal_pars,
               'ROC_AUC': svm_info.optimum
               })

Optimal parameters{'kernel': 'rbf', 'C': 4.973967190410129, 'sigfall_class_weight': 6.765950520833336, 'coef0': None, 'degree': None, 'logGamma': -3.4251259403278747}
AUROC of tuned SVM: 0.729


In [56]:
# Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

solvers = {'solver': {'svd': { 'sigfall_prior': [0.01,0.99]}, 
                       'lsqr': { 'sigfall_prior': [0.01,0.99] }, 
                       'eigen': { 'sigfall_prior': [0.01,0.99] } }}

def lda_tuned_auroc(x_train, y_train, x_test, y_test, solver, sigfall_prior=0):
    no_fall_prior = 1.0 - sigfall_prior
    model = LinearDiscriminantAnalysis(solver=solver, priors=[no_fall_prior,sigfall_prior]).fit(x_train, y_train)    
    decision_values = model.predict(x_test)
    return optunity.metrics.roc_auc(y_test, decision_values)

lda_tuned_auroc = cv_decorator(lda_tuned_auroc)

lda_optimal_pars, lda_info, _ = optunity.maximize_structured(lda_tuned_auroc, search_space=solvers, num_evals=100)

print("Optimal parameters" + str(lda_optimal_pars))
print("AUROC of tuned LDA: %1.3f" % lda_info.optimum)

results.append({'model': 'Linear Discriminant Analysis',
               'Optimal parameters': lda_optimal_pars,
               'ROC_AUC': lda_info.optimum
               })

Optimal parameters{'solver': 'lsqr', 'sigfall_prior': 0.6107157879477079}
AUROC of tuned LDA: 0.672


In [58]:
# Quadratic Discriminant Analysis

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def qda_tuned_auroc(x_train, y_train, x_test, y_test, sigfall_prior):
    no_fall_prior = 1.0 - sigfall_prior
    model = QuadraticDiscriminantAnalysis(priors=[no_fall_prior,sigfall_prior]).fit(x_train, y_train)
    decision_values = model.predict(x_test)
    auc = optunity.metrics.roc_auc(y_test, decision_values)
    return auc
    
    
qda_tuned_auroc = cv_decorator(qda_tuned_auroc)
qda_optimal_pars, qda_info, _ = optunity.maximize(qda_tuned_auroc, solver_name='grid search', num_evals=100, sigfall_prior=[0.01,0.99])

print("Optimal parameters" + str(qda_optimal_pars))
print("AUROC of tuned model: %1.3f" % qda_info.optimum)

results.append({'model': 'Quadratic Discriminant Analysis',
               'Optimal parameters': qda_optimal_pars,
               'ROC_AUC': qda_info.optimum
               })

Optimal parameters{'sigfall_prior': 0.8871}
AUROC of tuned model: 0.677


In [ ]:
# Random Forest (Decision Tree)

from sklearn.ensemble import RandomForestClassifier

